In [91]:
 #这个版本为尝试版本，因为ml已经调到89了，接下来使用bert进行尝试
    # result_ = pd.DataFrame(test['sid'])
# result_['label'] = res
# result_.to_csv('./baseline_xgboost_7z.csv', index=False)
# result_

In [92]:
 # Q1  是否能够embedding 特征表 
 # Q2   是否能够接 bert 
# Q3  是否能够 crf+ MLP 作为decoder 

In [93]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore') # 不会报warning错误

# 数据加载
train = pd.read_csv('./train.csv')
test = pd.read_csv('./test1.csv')
test = test.iloc[:, 1:]
train = train.iloc[:, 1:]

In [94]:
features = train.columns.tolist()
features.remove('label')

In [95]:

#加 osv
train['fea_hash'].map(lambda x: len(str(x))).value_counts()
train['fea1_hash'].map(lambda x: len(str(x))).value_counts()
remove_list = ['os','sid']
col = features
for i in remove_list:
    col.remove(i)
features = train[col]
features['fea_hash_len'] = features['fea_hash'].map(lambda x: len(str(x)))
features['fea1_hash_len'] = features['fea1_hash'].map(lambda x: len(str(x)))
features['fea_hash'] = features['fea_hash'].map(lambda x: 0 if len(str(x))>16 else int(x))
features['fea1_hash'] = features['fea1_hash'].map(lambda x: 0 if len(str(x))>16 else int(x))
test_features = test[col]
test_features['fea_hash_len'] = test_features['fea_hash'].map(lambda x: len(str(x)))
test_features['fea1_hash_len'] = test_features['fea1_hash'].map(lambda x: len(str(x)))
test_features['fea_hash'] = test_features['fea_hash'].map(lambda x: 0 if len(str(x))>16 else int(x))
test_features['fea1_hash'] = test_features['fea1_hash'].map(lambda x: 0 if len(str(x))>16 else int(x))
def app_osv(x):
    x=str(x).replace('Android_','').replace(' ','').replace('.','').replace('60十核20G_HD','').replace('Android','').replace('44W','')
    #
    if x=='nan' or x=='GIONEE_YNGA'or x=='f073b_changxiang_v01_b1b8_20180915' or x==''or x=='%E6%B1%9F%E7%81%B5OS+50':
        return 810
    elif x.count('-')>0:
        return  int(x.split('-')[0])
    else:
        res=int(x)
        if res<10:
            return res*100
        elif res<100:
            return res*10
    return res
features['osv']=features['osv'].apply(app_osv)
test_features['osv']=test_features['osv'].apply(app_osv)           

In [96]:
# 加timestamp
from datetime import datetime
features['timestamp'] = features['timestamp'].apply(lambda x : datetime.fromtimestamp(x/1000))
# 分解时间
# features['year'] = features['timestamp'].dt.year
# features['month'] = features['timestamp'].dt.month
# features['day'] = features['timestamp'].dt.day
# features['weekday'] = features['timestamp'].dt.weekday
# features['hour'] = features['timestamp'].dt.hour
# features['minute'] = features['timestamp'].dt.minute
start_time = features['timestamp'].min()
features['timestamp_diff_start'] = features['timestamp']-start_time
features['timestamp_diff_start'] = features['timestamp_diff_start'].dt.days - features['timestamp_diff_start'].dt.seconds/3600/24
test_features['timestamp'] = test_features['timestamp'].apply(lambda x : datetime.fromtimestamp(x/1000))
# 分解时间
# test_features['year'] = test_features['timestamp'].dt.year
# test_features['month'] = test_features['timestamp'].dt.month
# test_features['day'] = test_features['timestamp'].dt.day
# test_features['weekday'] = test_features['timestamp'].dt.weekday
# test_features['hour'] = test_features['timestamp'].dt.hour
# test_features['minute'] = test_features['timestamp'].dt.minute
start_time = test_features['timestamp'].min()
test_features['timestamp_diff_start'] = test_features['timestamp']-start_time
test_features['timestamp_diff_start'] = test_features['timestamp_diff_start'].dt.days - test_features['timestamp_diff_start'].dt.seconds/3600/24

In [97]:
def trans_version(version):
    version = version.replace('V','').replace('P_Final_','').replace(' ','').replace('GA','').replace('v','')
    if version == '50':
        return int(5)
    return int(version)
features['version'] = features['version'].apply(trans_version)
test_features['version'] = test_features['version'].apply(trans_version)

In [98]:
def trans_lan(lan):
    lan = str(lan)
    if 'zh' in lan.lower() or 'cn' in lan.lower():
        return int(0)
    else:
        return int(1)
features['lan'] = features['lan'].apply(trans_lan)
test_features['lan']=test_features['lan'].apply(trans_lan)

In [99]:
features['osv1'] = features['osv'] - features['version'] 
features['dev_area'] = features['dev_height']*features['dev_width']
test_features['osv1'] = test_features['osv'] - test_features['version'] 
test_features['dev_area'] =test_features['dev_height']*test_features['dev_width'] 

In [100]:
features=features.drop(['timestamp'], axis=1)
test_features=test_features.drop(['timestamp'], axis=1)

In [101]:
# class Bert_block(dygraph.layers.Layer):
#     def __init__(self):
#         super(Model, self).__init__()
#         # 离散特征做Embedding
#         self.apptype_emb = Embedding([89 + 1, 728], is_sparse=False)
#         self.apptype_fc = Linear(32, 32)

#         self.carrier_emb = Embedding([5 + 1, 32], is_sparse=False)
#         self.carrier_fc = Linear(32, 32)

#         self.dev_height_emb = Embedding([864 + 1, 512], is_sparse=False)
#         self.dev_height_fc = Linear(32, 32)

#         self.dev_ppi_emb = Embedding([105 + 1, 32], is_sparse=False)
#         self.dev_ppi_fc = Linear(32, 32)

#         self.dev_width_emb = Embedding([382 + 1, 32], is_sparse=False)
#         self.dev_width_fc = Linear(32, 32)

#         self.lan_emb = Embedding([25 + 1, 32], is_sparse=False)
#         self.lan_fc = Linear(32, 32)

#         self.media_id_emb = Embedding([292 + 1, 32], is_sparse=False)
#         self.media_id_fc = Linear(32, 32)

#         self.ntt_emb = Embedding([8 + 1, 32], is_sparse=False)
#         self.ntt_fc = Linear(32, 32)

#         self.osv_emb = Embedding([165 + 1, 32], is_sparse=False)
#         self.osv_fc = Linear(32, 32)

#         self.package_emb = Embedding([2102 + 1, 64], is_sparse=False)
#         self.package_fc = Linear(64, 64)

#         self.version_emb = Embedding([23 + 1, 16], is_sparse=False)
#         self.version_fc = Linear(16, 16)

#         self.location_emb = Embedding([332 + 1, 16], is_sparse=False)
#         self.location_fc = Linear(16, 16)

#         self.fea1_hash_emb = Embedding([6147 + 1, 64], is_sparse=False)
#         self.fea1_hash_fc = Linear(64, 64)

#         self.dense_fc = Linear(1, 32)

#         self.fc1 = Linear(416 + 128-64 , 800, act='relu')
#         self.fc2 = Linear(800, 900, act='relu')
#         self.fc3 = Linear(900, 900, act='relu')
#         self.fc4 = Linear(900, 2, act='softmax') 

# embedding 

In [102]:
df=pd.concat([features, test_features])

In [103]:
for i in df.columns:
      print('{}存在着{}个不同的值'.format(i,len(set(df[i]))))

android_id存在着467958个不同的值
apptype存在着89个不同的值
carrier存在着5个不同的值
dev_height存在着864个不同的值
dev_ppi存在着105个不同的值
dev_width存在着382个不同的值
lan存在着2个不同的值
media_id存在着292个不同的值
ntt存在着8个不同的值
osv存在着108个不同的值
package存在着2102个不同的值
version存在着14个不同的值
fea_hash存在着509360个不同的值
location存在着332个不同的值
fea1_hash存在着6147个不同的值
cus_type存在着58个不同的值
fea_hash_len存在着16个不同的值
fea1_hash_len存在着5个不同的值
timestamp_diff_start存在着377060个不同的值
osv1存在着236个不同的值
dev_area存在着1169个不同的值


## 分析 

- android_id、sid、 不需要做Embedding
- fea_hash  fea1_hash timestamp_diff_start  norm    
- os同一个值，不需要加入计算

In [104]:
embed_list = ['apptype','carrier','dev_height','dev_ppi','dev_width','lan','media_id','ntt','osv','package','version','location','cus_type','fea_hash_len','fea1_hash_len','osv1','dev_area']
len(embed_list)


17

In [105]:
import pickle

In [106]:


def save_obj(obj, name):
    with open(name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)
 
 
def load_obj(name):
    with open(name + '.pkl', 'rb') as f:
        return pickle.load(f)



def gen_embed_vocab(data, col, vocab_path):
    data_hash = dict()
    # 制作字典
    for sample in data:
        sample = str(sample)
        if sample not in data_hash:
            data_hash[sample] = len(data_hash) + 1
    # 保存字典
    save_obj(data_hash, vocab_path+col+'_vocab')
    print(col, "字典生成完毕,共", len(data_hash), "个id")
    return data_hash


for col in embed_list:
     gen_embed_vocab(df[col], col, 'vocab/')

apptype 字典生成完毕,共 89 个id
carrier 字典生成完毕,共 5 个id
dev_height 字典生成完毕,共 864 个id
dev_ppi 字典生成完毕,共 105 个id
dev_width 字典生成完毕,共 382 个id
lan 字典生成完毕,共 2 个id
media_id 字典生成完毕,共 292 个id
ntt 字典生成完毕,共 8 个id
osv 字典生成完毕,共 108 个id
package 字典生成完毕,共 2102 个id
version 字典生成完毕,共 14 个id
location 字典生成完毕,共 332 个id
cus_type 字典生成完毕,共 58 个id
fea_hash_len 字典生成完毕,共 16 个id
fea1_hash_len 字典生成完毕,共 5 个id
osv1 字典生成完毕,共 236 个id
dev_area 字典生成完毕,共 1169 个id


##  构建dataset以及 dataloader

In [107]:

# 各字段网络类型
NETWORK_TAGS = {'android_id': None,
                'apptype': "emb",
                'carrier': "emb",
                'dev_height': "emb",
                'dev_ppi': "emb",
                'dev_width': "emb",
                'lan': "emb",
                'media_id': "emb",
                'ntt': "emb",
                'osv': "emb",
                'package': "emb",
                'timestamp_diff_start': "norm",
                'version': "emb",
                'fea_hash': "norm",
                'location': "emb",
                'fea1_hash': "norm",
                'cus_type': "emb",
               'fea_hash_len':"emb",
               'fea1_hash_len':"emb",
               'osv1':"emb",
               'dev_area':"emb"}

In [108]:
# 需要embedding的大小
emb_size = {}
# 训练字段
FEATURES_ = [] 

In [109]:
from sklearn.preprocessing import StandardScaler

In [110]:
# 对不需要embedding进行归一化处理
unemb_cols = []
for k, v in NETWORK_TAGS.items():
    if v == 'norm':
        unemb_cols.append(k)

In [111]:
unemb_cols

['timestamp_diff_start', 'fea_hash', 'fea1_hash']

In [112]:
scaler = StandardScaler()
scaler_data = scaler.fit_transform(df[unemb_cols])
df[unemb_cols] = scaler_data

In [113]:
df[unemb_cols] 

,timestamp_diff_start,fea_hash,fea1_hash
0,0.408626,0.022349,0.022888
1,0.832981,0.544812,0.455496
2,-0.284325,-0.576734,-1.352032
3,1.573846,1.174594,-0.822602
4,0.559519,0.207594,-0.639184
...,...,...,...
149995,1.053318,0.852001,0.331627
149996,0.585310,-1.622461,0.304501
149997,1.577644,-0.635404,0.250249
149998,-0.426720,-0.657304,-0.325912


In [114]:
# embedding_ 
def load_vocab(name):
    with open('vocab/' + name + '_vocab.pkl', 'rb') as f:
        return pickle.load(f)

for k, v in NETWORK_TAGS.items():
    if v == 'emb':
        emb_dict = load_vocab(k)
        emb_size[k] = len(emb_dict)
        df[k] = df[k].astype('str').map(emb_dict)
    if v is not None:
        FEATURES_.append(k)

In [127]:
features_=df[:500000]
test_features_=df[500000:]

In [ ]:
class MineDataset(Dataset):

    def __init__(self, data: pd.DataFrame, features: list, tags: dict):
        super(MineDataset, self).__init__()
        self.len = len(data)
        self.features = features
        self.tags = tags
        self.X = data[features]
        self.y = data['label']

    def __getitem__(self, idx):
        result = []
        tmp_x = self.X.iloc[idx]
        # 转换数据类型
        for k, v in self.tags.items():
            if v == 'emb':
                result.append(np.array(tmp_x[k], dtype='int64').reshape([1]))
            elif v == 'norm':
                result.append(np.array(tmp_x[k], dtype='float32').reshape([1]))
        # 如果是预测，添加标签
        label = self.y.iloc[idx]
        if label != -1:
            result.append(np.array(label).astype("int64"))

        return result

    def __len__(self):
        return self.len